In [ ]:
# import ir_datasets
# dataset = ir_datasets.load("msmarco-passage")
# for doc in dataset.docs_iter():
#     doc # namedtuple<doc_id, text>

In [37]:
from elasticsearch import Elasticsearch
import pandas as pd
from sentence_transformers import CrossEncoder
from typing import Callable

In [38]:
es = Elasticsearch()
# es.info()
INDEX_NAME = "passage_index"
doc = es.get(index=INDEX_NAME, id=1)
print(doc)

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x000001DE42CC1600>: Failed to establish a new connection: [WinError 10061] Kan ikke koble til fordi målmaskinen avslo tilkobling) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x000001DE42CC1600>: Failed to establish a new connection: [WinError 10061] Kan ikke koble til fordi målmaskinen avslo tilkobling)

In [42]:
queries_eval = pd.read_csv("data/queries/queries.eval.tsv", sep='\t', header=None)#, index_col=0)
queries_eval.head()

,0,1
0,786436,what is prescribed to treat thyroid storm
1,9,Refer to the data. Diminishing returns begin ...
2,786450,what is presentation software?
3,524308,treasury routing number
4,33,game called poem who wrote what occasion


In [46]:
collection_df = pd.read_csv("data/collection/collection.tsv", sep='\t', header=None) #, index_col=0)
print(len(collection_df))
collection_df.head()

8841823


,0,1
0,0,The presence of communication amid scientific ...
1,1,The Manhattan Project and its atomic bomb help...
2,2,Essay on The Manhattan Project - The Manhattan...
3,3,The Manhattan Project was the name for a proje...
4,4,versions of each volume as well as complementa...


In [50]:
qrelspath = "data/queries/qrels.txt"

qrels_ids = set()
with open(qrelspath, encoding="utf-8") as file:
    for line in file:
        l = line.split(' ')
        qrels_ids.add(l[0])

In [ ]:
# def relevant_queries(queries, qrels):
#     relevant_queries = []
#     relevant_queries_id = []

#     for idx, query in enumerate(queries):
#         query_id = str(queries_id[idx])
#         if query_id in qrels:
#             relevant_queries.append(query)
#             relevant_queries_id.append(query_id)

#     return relevant_queries, relevant_queries_id

In [53]:
queriesToUse = {}
for _, query in queries_eval.iterrows():
    if str(query[0]) in qrels_ids:
        queriesToUse[query[0]] = query[1]

In [ ]:
# model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=512)

# ranked = {}

# # Rank the top 1000 passages for each query
# for query_id, query_text in queriesToUse.items():
#     # print(query_id, query_text)
#     # break
#     for _, doc in collection_df.iterrows():
#         # print(doc[0], doc[1])
#         # break
#         doc_id = doc[0]
#         doc_text = doc[1]
#         score = model.predict([(query_text, doc_text)])[0]
#         if query_id not in ranked:
#             ranked[query_id] = []
#         ranked[query_id].append((doc_id, score))
#         break
#     ranked[query_id] = sorted(ranked[query_id], key=lambda x: x[1], reverse=True)[:1000]
#     break
# print(ranked)

In [141]:
query_topK = {}
for query_id, query in queriesToUse.items():
    # query = queries[idx]
    res = es.search(index=INDEX_NAME, q=query, _source=False, size=5000, request_timeout=60)
    top_k_scores = [hit["_id"] for hit in res["hits"]["hits"]]
    query_topK[str(query_id)] = top_k_scores

c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [147]:
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=512)

ranked = {}

# Rank the top 5000 passages for each query
for i, (query_id, doc_ids) in enumerate(query_topK.items()):
    print(f"Processed: {round(i/len(query_topK)*100,2)}%")
    # print(query_id, doc_ids)
    # print(es.get(index=INDEX_NAME, id=doc_ids[0])["_source"]["content"])
    # print(queriesToUse[query_id])

    l = [(queriesToUse[int(query_id)], es.get(index=INDEX_NAME, id=doc_id)["_source"]["content"]) for doc_id in doc_ids]
    score = model.predict(l)
    # print(score)
    # print(len(score))
    d = dict(zip(doc_ids, score))
    ranked[query_id] = [k for k in sorted(d, key=d.get, reverse=True)]
#     break
# print(ranked)

Processed: 0.0%


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Processed: 2.33%
Processed: 4.65%
Processed: 6.98%
Processed: 9.3%
Processed: 11.63%
Processed: 13.95%
Processed: 16.28%
Processed: 18.6%
Processed: 20.93%
Processed: 23.26%
Processed: 25.58%
Processed: 27.91%
Processed: 30.23%
Processed: 32.56%
Processed: 34.88%
Processed: 37.21%
Processed: 39.53%
Processed: 41.86%
Processed: 44.19%
Processed: 46.51%
Processed: 48.84%
Processed: 51.16%
Processed: 53.49%
Processed: 55.81%
Processed: 58.14%
Processed: 60.47%
Processed: 62.79%
Processed: 65.12%
Processed: 67.44%
Processed: 69.77%
Processed: 72.09%
Processed: 74.42%
Processed: 76.74%
Processed: 79.07%
Processed: 81.4%
Processed: 83.72%
Processed: 86.05%
Processed: 88.37%
Processed: 90.7%
Processed: 93.02%
Processed: 95.35%
Processed: 97.67%


# Evaluation

In [133]:
# Bulk indexing
qrelspath = "data/qrels/qrels.txt"

qrels = {}
with open(qrelspath, encoding="utf-8") as file:
    for line in file:
        l = line.split(' ')

        qid = l[0]
        pid = l[2]
        relevance = int(l[3])

        if relevance > 0:
            if qid in qrels.keys():
                qrels[qid].add(pid)
            else:
                qrels[qid] = set([pid])

In [134]:
def get_average_precision(system_ranking, ground_truth) -> float:
    vals = []
    over = 1
    for rank_idx, rank in enumerate(system_ranking):
        under = rank_idx+1
        if rank in ground_truth:
            vals.append(over / under)
            over += 1
    AP = sum(vals) / len(ground_truth)

    return AP

In [148]:
system_ranking = query_topK["527433"] # List
system_truth = qrels["527433"] # Set
score = get_average_precision(system_ranking, system_truth)
score

0.09030257145979574

In [136]:
def get_reciprocal_rank(system_ranking, ground_truth) -> float:
    AP = 0
    for rank_idx, rank in enumerate(system_ranking):
        under = rank_idx+1
        if rank in ground_truth:
            AP = 1 / under
            break
    
    return AP

In [149]:
system_ranking = query_topK["527433"] # List
system_truth = qrels["527433"] # Set
score = get_reciprocal_rank(system_ranking, system_truth)
score

1.0

In [138]:
def get_mean_eval_measure(system_rankings, ground_truths, eval_function: Callable) -> float:
    results = []
    for query in system_rankings:
        if query in ground_truths.keys():
            results.append(eval_function(system_rankings[query], ground_truths[query]))
        else:
            continue
            # results.append(0) -> ?
    return sum(results) / len(results)

In [150]:
map = get_mean_eval_measure(query_topK, qrels, get_average_precision)
mrr = get_mean_eval_measure(query_topK, qrels, get_reciprocal_rank)

## From BM25
map = 0.32872575816078825

mrr = 0.7265016684853105

In [152]:
print(map)
print(mrr)

0.3324887414761848
0.7265135518853889
